In [ ]:
library(readxl)
library(dplyr)
library(ggplot2)
library(rstatix)
library(tidyr)

# Load the data
data <- read_excel("data_20240522_B.xlsx")


# Zobrazení prvních několika řádků data frame
data$zlepseni <- (data$emise_pred - data$emise_po) / data$emise_pred * 100

boxplot(data$zlepseni ~ data$znacka, xlab = "Značka", ylab = "Emise před seřízením", col = "lightblue")

outliers <- data %>%
    group_by(znacka) %>%
    rstatix::identify_outliers(zlepseni)

outliers

data$zlepseni <- ifelse(data$id %in% outliers$id, NA, data$zlepseni)
data <- na.omit(data)
boxplot(data$zlepseni ~ data$znacka, xlab = "Značka", ylab = "Emise před seřízením", col = "lightblue")

summry <- data %>%
    group_by(znacka) %>%
    summarise(
        mean = mean(zlepseni),
        median = median(zlepseni),
        min = min(zlepseni),
        max = max(zlepseni),
        sd = sd(zlepseni),
        norm_test = shapiro.test(zlepseni)$p.value,
        sym_test = lawstat::symmetry.test(zlepseni, boot = FALSE)$p.value
    )
summry
bartlett.test(zlepseni ~ znacka, data = data)
kruskal.test(zlepseni ~ znacka, data = data)
dunn.test::dunn.test(data$zlepseni, data$znacka, method = "bonferroni", altp = TRUE)

data <- data


In [ ]:
res = aov(zlepseni ~ znacka, data = data)
summary(res)
TukeyHSD(res)

In [ ]:
t.test(data$zlepseni[data$znacka == "X"], conf.level = 0.95)

In [ ]:
table(data$zlepseni[data$znacka == "X"] > 12)
binom.test(x = 0,n = 146, alternative = "less")

In [ ]:
var.test(data$zlepseni[data$znacka == "X"], data$zlepseni[data$znacka == "Y"],ratio = 1)

In [ ]:
t.test(data$zlepseni[data$znacka == "X"], data$zlepseni[data$znacka == "Y"], alternative = "two.sided", conf.level = 0.95)
13.48434 - 10.98947

In [ ]:
autox_horsi = data$zlepseni[data$znacka == "X"] < 10
table(autox_horsi)
auto_x_zavada = data$problem[data$znacka == "X"]
table(auto_x_zavada)
tab = table(auto_x_zavada, autox_horsi)
tab = tab[c(2, 1), c(2, 1)]
tab

In [ ]:
mosaicplot(tab, color = c("red", "blue"), main = "Vztah mezi závadou a zhoršením emisí")
cramer_v(tab)

In [ ]:
epiR::epi.2by2(tab)